In [1]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install python-dotenv --quiet
import dotenv
import os
dotenv.load_dotenv(
        os.path.join('/content/drive/MyDrive/vscode-ssh', '.env')
    )
password = os.getenv('PASSWORD')
github_access_token = os.getenv('GITHUB_ACCESS_TOKEN')

In [ ]:
!pip install colab_ssh --upgrade --quiet
from colab_ssh import launch_ssh_cloudflared, init_git_cloudflared

In [ ]:
launch_ssh_cloudflared(password)
init_git_cloudflared(repository_url=git_repo + ".git",
         personal_token=github_access_token, 
         branch="main",
         email="<email_for_github>",
         username="<github_username>")

In [ ]:
from text2brick.dataset import LegoDatasetGenerator
from text2brick.gym import IoUValidityRewardFunc
reward_function = IoUValidityRewardFunc()
generator = LegoDatasetGenerator(reward_function=reward_function)

In [2]:
generator.generate_dataset(10)

100%|██████████| 10/10 [00:05<00:00,  1.76it/s]


In [1]:
from text2brick.dataset import LegoPretrainDataset
from torch_geometric.loader import DataLoader 
dataset = LegoPretrainDataset("./lego_dataset")
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

c:\Users\ebern\Documents\GitHub\Text2Bricks\text2brick\dataset\LegoDataset.py:25: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(file_path)


In [2]:
from text2brick.gym import Text2Brick_v1
model = Text2Brick_v1()

Using cache found in C:\Users\ebern/.cache\torch\hub\pytorch_vision_v0.10.0


In [5]:
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

# Define the loss function (CrossEntropyLoss for classification tasks)
x_loss_fn = nn.CrossEntropyLoss()
y_loss_fn = nn.CrossEntropyLoss()

# Define optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-3)

def train_model(model, dataloader, optimizer, x_loss_fn, y_loss_fn, num_epochs=10):
    model.train()  # Set the model to training mode

    for epoch in range(num_epochs):
        epoch_loss = 0.0
        
        # Loop over the batches of data
        for batch in tqdm(dataloader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
            optimizer.zero_grad()  # Zero the parameter gradients

            image_target = batch[0]
            current_build_img = batch[1]
            next_node = batch[2]
            reward = batch[3]
            current_graph = batch[4]
            
            x_pred, y_pred = model(current_build_img, current_graph, reward, image_target=image_target, return_logits=True)
            x_true, y_true = next_node[:, 0], next_node[:, 1]

            # Compute loss for both x and y coordinates
            x_loss = x_loss_fn(x_pred, x_true)
            y_loss = y_loss_fn(y_pred, y_true)

            # Combine the losses (you can weight them differently if needed)
            loss = x_loss + y_loss
            
            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        # Print epoch loss
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}")

# Run the training loop
train_model(model, dataloader, optimizer, x_loss_fn, y_loss_fn, num_epochs=10)


Epoch 1/10:   0%|          | 1/252 [00:00<01:03,  3.94it/s]

x shape before gcn1: torch.Size([39, 2])
edge_index shape before gcn1: torch.Size([2, 78])
torch.float32 torch.float32
tensor([15,  6]) tensor([10,  7])


Epoch 1/10:   1%|          | 2/252 [00:00<00:55,  4.50it/s]

x shape before gcn1: torch.Size([21, 2])
edge_index shape before gcn1: torch.Size([2, 38])
torch.float32 torch.float32
tensor([12,  5]) tensor([13,  1])


Epoch 1/10:   1%|          | 3/252 [00:00<00:57,  4.36it/s]

...

Epoch 1/10:  30%|██▉       | 75/252 [00:17<00:48,  3.64it/s]

x shape before gcn1: torch.Size([60, 2])
edge_index shape before gcn1: torch.Size([2, 134])
torch.float32 torch.float32
tensor([13, 16]) tensor([18,  1])


Epoch 1/10:  30%|███       | 76/252 [00:17<00:40,  4.36it/s]

x shape before gcn1: torch.Size([61, 2])
edge_index shape before gcn1: torch.Size([2, 132])
torch.float32 torch.float32
tensor([13, 20]) tensor([ 6, 16])


KeyError: 'saved'